In [2]:
# using example from https://keras.io/examples/nlp/addition_rnn/
from typing import List

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Parameters for the model and dataset.
TRAINING_SIZE: int = 50_000
DIGITS: int = 3
MAX_NUMBER_OF_ELEMENTS: int = 3
REVERSE: bool = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN: int = MAX_NUMBER_OF_ELEMENTS * DIGITS + 2 * MAX_NUMBER_OF_ELEMENTS

In [3]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars: str):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = {c: i for i, c in enumerate(self.chars)}
        self.indices_char = {i: c for i, c in enumerate(self.chars)}

    def encode(self, C: str, num_rows: int):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x: np.array, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


chars = "0123456789, "
ctable = CharacterTable(chars)

## Generate trainig data

In [4]:
def make_random_number() -> int:
    return int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )


def make_unified_lenght(string: str) -> str:
    return string + " " * (MAXLEN - len(string))


def list_to_string(list_to_convert: List[int]) -> str:
    return ", ".join([str(element) for element in list_to_convert])


questions: List[str] = []
answers: List[str] = []

print("Generating data...")
while len(questions) < TRAINING_SIZE:
    unsorted_list = []
    for element_idx in range(MAX_NUMBER_OF_ELEMENTS):
        unsorted_list.append(make_random_number())
    sorted_list = sorted(unsorted_list)
    unsorted_list_as_string, sorted_list_as_string = list_to_string(
        unsorted_list
    ), list_to_string(sorted_list)
    query = make_unified_lenght(unsorted_list_as_string)
    answer = make_unified_lenght(sorted_list_as_string)
    if query not in questions:
        questions.append(query)
        answers.append(answer)
print("Total questions:", len(questions))

Generating data...
Total questions: 50000


In [5]:
questions[:10]

['28, 488, 687   ',
 '0, 90, 546     ',
 '399, 3, 80     ',
 '8, 881, 37     ',
 '46, 791, 7     ',
 '65, 124, 5     ',
 '5, 61, 375     ',
 '0, 4, 82       ',
 '1, 785, 49     ',
 '25, 900, 399   ']

In [6]:
answers[:10]

['28, 488, 687   ',
 '0, 90, 546     ',
 '3, 80, 399     ',
 '8, 37, 881     ',
 '7, 46, 791     ',
 '5, 65, 124     ',
 '5, 61, 375     ',
 '0, 4, 82       ',
 '1, 49, 785     ',
 '25, 399, 900   ']

## Vectorize the data

In [7]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(answers):
    y[i] = ctable.encode(sentence, MAXLEN)

Vectorization...


In [8]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 15, 12)
(45000, 15, 12)
Validation Data:
(5000, 15, 12)
(5000, 15, 12)


## Build the model

In [9]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(256, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(MAXLEN))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...


2022-04-21 21:06:01.617232: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               275456    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 15, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 15, 128)           197120    
_________________________________________________________________
dense (Dense)                (None, 15, 12)            1548      
Total params: 474,124
Trainable params: 474,124
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [10]:
epochs = 20
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("✅ " + guess)
        else:
            print("❌ " + guess)


Iteration 1


2022-04-21 21:06:02.800815: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


1407/1407 [==============================] - 55s 36ms/step - loss: 0.9009 - accuracy: 0.6629 - val_loss: 0.5878 - val_accuracy: 0.7482
Q 68, 5, 1        T 1, 5, 68        ❌ 1, 6, 81       
Q 603, 8, 6       T 6, 8, 603       ❌ 0, 6, 666      
Q 9, 155, 8       T 8, 9, 155       ❌ 5, 5, 555      
Q 195, 76, 59     T 59, 76, 195     ❌ 55, 55, 999    
Q 0, 731, 17      T 0, 17, 731      ❌ 1, 10, 771     
Q 769, 1, 852     T 1, 769, 852     ❌ 1, 189, 989    
Q 96, 2, 17       T 2, 17, 96       ❌ 2, 22, 92      
Q 69, 7, 5        T 5, 7, 69        ❌ 5, 9, 99       
Q 9, 8, 99        T 8, 9, 99        ❌ 9, 9, 99       
Q 0, 58, 79       T 0, 58, 79       ❌ 0, 70, 90      

Iteration 2
1407/1407 [==============================] - 49s 35ms/step - loss: 0.5554 - accuracy: 0.7605 - val_loss: 0.5459 - val_accuracy: 0.7662
Q 0, 548, 234     T 0, 234, 548     ❌ 4, 422, 844    
Q 731, 7, 26      T 7, 26, 731      ❌ 2, 22, 776     
Q 3, 8, 91        T 3, 8, 91        ❌ 3, 8, 83       
Q 99, 5, 13    